# Object Localization Demonstration Notebook
### Import statements

In [1]:
import pyrealsense2 as rs
import numpy as np
import time
# TODO import trained YOLOv4 object identification model

### Declaring constant variables, obtained from values in Intel and VRC documentation

In [2]:
focal_length = ((448.0-172.0) * 24.0) / 11.0
width_dict = {"Ring":3.5, "Neutral_Goal":12.5, "Blue_Goal":12.5, "Red_Goal":12.5, "Blue_Robot":5.5, "Red_Robot":5.5, "Blue_Platform":53.0, "Red_Platform":53.0}

### Declaring and initializing trained YOLOv4 model

In [3]:
# TODO declare YOLOv4 model

### Starting image pipeline from D435 camera

In [4]:
pipeline = rs.pipeline()
pipeline.start()

RuntimeError: No device connected

### Object localization method

In [5]:
def obj_distance(obj):
    # kitti file parsing
    obj_array = obj.split()
    x_min, y_min, x_max, y_max = float(obj_array[4]), float(obj_array[5]), float(obj_array[6]), float(obj_array[7])
    
    # calculating centroid of object
    centroid_x, centroid_y = int((x_min+x_max)/2.), int((y_min+y_max)/2.)
    
    # calculating distance using trigonometric properties
    trig_distance = (width_dict[obj_array[0]] * focal_length)/(x_max - x_min) 
    
    # extract distance from depth map and convert to inches
    depth_distance_meters = depth_frame.get_distance(centroid_x, centroid_y) 
    depth_distance = 39.3701 * depth_distance_meters
    
    # weighting and combining localization methods
    distance = (trig_distance * .2) + (depth_distance_meters * .8) 
    
    # in the event that depthmap can't detect distance, only use trig distance
    if (depth_distance_meters == 0):
        distance = trig_distance
    
    return distance

### Using identification and localization to process images from the pipeline

In [6]:
while True:
    # extracting data from the image pipeline
    frames = pipeline.wait_for_frames()
    depth_frame = frames.get_depth_frame()
    color_frame = frames.get_color_frame()
    color_image = np.asanyarray(color_frame.get_data())
    
    # TODO add IO for trained YOLOv4 model (color_img->img_kitti_file)
    
    # reading labels from kitti file
    game_objects = []
    with open('img_kitti.txt') as f:
        game_objects = f.readlines()   
    
    # calculating distance for all game objects in frame
    for obj in game_objects:
        print(f'{obj.split()[0]}: {obj_distance(obj)})')
    
    # closing the identification models output file
    f.close()
    # 0.1 second buffer to prevent pipeline from bottlenecking
    time.sleep(.1) 

RuntimeError: wait_for_frames cannot be called before start()